<a href="https://colab.research.google.com/github/zjwster/Clash.Meta/blob/Meta/%E2%80%9C%E8%82%A1%E7%A5%A803240924_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ta
#!pip install yfinance
!pip install matplotlib
#!pip install datetime
!pip install tushare
!pip install pinyin
#!pip install mpl_finance
!pip install requests
!pip install io

In [ ]:
import ta
import tushare as ts
import pinyin
import matplotlib.pyplot as plt
import requests
import io

# 填写您的 Rocket.Chat 服务器地址和用户凭证
rocketchat_url = 'https://icenter.tech'
rocketchat_user_id = 'GebgaWx7PBTF3aa6n'
rocketchat_auth_token = 'QRJpOkLDplm3NQ_EP83lqiCjLTmMVi21OsBClnzIhzn'

ts.set_token('062244397b854ccce9496ffcc980dc7ab68b4e480981ba7fd7146117')
names = {'300738': 'aofeishuju', '603881': 'shujugang', '300520': 'kedaguochuang', '600036': 'zhaoshangyinhang', '002916': 'shennandianlu'}

for code, name in names.items():
    # 获取股票数据
    df = ts.get_k_data(code, start='2023-01-01')

    # 计算布林带指标
    bb = ta.volatility.BollingerBands(df['close'])
    df['BB_upper'] = bb.bollinger_hband()
    df['BB_middle'] = bb.bollinger_mavg()
    df['BB_lower'] = bb.bollinger_lband()

    # 计算趋势线
    trendline_x = [0, len(df)]
    trendline_y = [df['close'][0], df['close'][len(df) - 1]]

    # 计算支撑/阻力线
    resistance_x = [0, len(df)]
    resistance_y = [df['close'].max(), df['close'].max()]
    support_x = [0, len(df)]
    support_y = [df['close'].min(), df['close'].min()]

    # 计算相对强弱指数
    rsi = ta.momentum.RSIIndicator(df['close'])
    df['RSI'] = rsi.rsi()

    # 将股票名称转换为拼音
    name_pinyin = pinyin.get(name, format='strip', delimiter='')

    # 绘制布林带图、趋势线、支撑/阻力线和相对强弱指数
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
    ax1.plot(df['close'])
    ax1.plot(df['BB_upper'], label='Upper Band')
    ax1.plot(df['BB_middle'], label='Middle Band')
    ax1.plot(df['BB_lower'], label='Lower Band')
    ax1.plot(trendline_x, trendline_y, label='Trendline')
    ax1.plot(resistance_x, resistance_y, label='Resistance')
    ax1.plot(support_x, support_y, label='Support')
    ax1.legend(loc='upper left')
    ax1.set_title(f'{name} ({code}) Bollinger Bands')
    ax2.plot(df['RSI'])
    ax2.set_title(f'{name} ({code}) RSI')
    ax2.axhline(y=30, color='r', linestyle='--')
    ax2.axhline(y=70, color='r', linestyle='--')
    plt.savefig(f'/content/drive/MyDrive/stock_chart/{name_pinyin}_{code}.png')

    # 将生成的图片上传到 Rocket.Chat
    with open(f'{name_pinyin}_{code}.png', 'rb') as f:
        data = io.BytesIO(f.read())
    response = requests.post(
        rocketchat_url + '/api/v1/rooms.upload/' + rocketchat_user_id,
        #rocketchat_url + '/api/v1/rooms.upload',
        # 'https://icenter.tech/api/v1/upload',
        # 'https://icenter.tech/api/v1/chat.postMessage',
        headers={
            'X-Auth-Token': rocketchat_auth_token,
            'X-User-Id': rocketchat_user_id
        },
        files={
            'file': ('chart.png', data, 'image/png')
        },
        data={
            'channel': '@zjwster',
            'text': 'chart',
            'emoji': ':dog:',
            'alias': '小助理',
        }
    )
    # print(response.json())
    # 检查响应状态码
    if response.status_code == 200:
        print("图片已成功发送到 RocketChat 频道！")
    else:
        print("发送图片时出错。错误代码：", response.status_code)